# The main loop

In [ ]:
%websocketconnect 192.168.1.130:8266 --password horsetoe

In [ ]:
import os
os.listdir()

In [ ]:
import uasyncio as asyncio
import wifi
import send_data
import sensors
import gc
import collections.deque
import ntptime
import network
import utime
import machine
import power_status

print("running garbage collector")
gc.collect()  # clean up memory after imports

wifi_interface = network.WLAN(network.STA_IF)

while not wifi_interface.isconnected():
    wifi.connectWifi()

print('network config:', wifi_interface.ifconfig())

data_queue = collections.deque.deque()
get_ID = send_data.generateMessageID()

print("setting time")
ntptime.settime()

def wdt_callback():
    global wdt_counter
    wdt_counter += 1
    print("wdt counter set to: {}".format(wdt_counter))
    if (wdt_counter > 10):
        try:
            print("watchdog timer has timed out")
            send_data.postData({'reset_time_date': list(utime.localtime()),
                                'reset_cause': 'watchdog_timer'}, 
                               url="http://192.168.1.151:9494/error")
        except Exception as e:
            print("Error sending wdt data: {}".format(e))
        finally:
            print("resetting device")
            machine.reset()

wdt_counter = 0
# Set a virtual timer by setting Timer ID of -1
wdt_timer = machine.Timer(-1)
# Create a periodic timer with period of 60000ms, aka 60 seconds
wdt_timer.init(period=60000, mode=machine.Timer.PERIODIC, callback=lambda t:wdt_callback())
print("wdt initialized")

# Reset codes:
# 0 = power on, 1 = WDT reset, 4 = soft reset, 5 = DEEP_SLEEP reset, 6 = hard reset,
def reportLastReset():
    try:
        print("Sending reset data")
        send_data.postData({'reset_time_date': list(utime.localtime()),
                            'reset_cause': machine.reset_cause()}, 
                            url="http://192.168.1.151:9494/error")
        print("Sent reset data to server")
    except Exception as e:
            print("Error sending reset data: {}".format(e))

reportLastReset()

loop_counter = 0

while True:
    print("loop counter = ", loop_counter)
    
    print("getting data")
    data_queue.append(sensors.readData())
    
    if loop_counter % 3 == 0:
        print("sending data")
        send_data.processQueue(data_queue, get_ID, "http://192.168.1.151:9494/station")
        print("feeding WatchDog timer")
        wdt_counter = 0

    if loop_counter % 6 == 0:
        print("checking Wifi Connection")
        wifi.connectWifi()
        print("sending battery status")
        send_data.postData(power_status.getBatteryStatus(),
                           url="http://192.168.1.151:9494/power")
    
    if loop_counter % 30 == 0:
        print("sending log data")
        #TODO send memory info, queue length
        
    if loop_counter % 60 == 0:
        print("collecting garbage")
        #http://docs.micropython.org/en/latest/reference/constrained.html#the-heap
        gc.collect()
        gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())
        
    if loop_counter % 2000 == 0:
        print("setting time with ntp")
        try:
            ntptime.settime()
        except OSError as e:
            print(e)

    loop_counter += 1
    if loop_counter > 2000:
        loop_counter = 0

    utime.sleep(10) # Consider deep sleep modes - https://docs.micropython.org/en/latest/library/esp.html

In [ ]:
import gc
gc.collect()

In [ ]:
main_script="""import uasyncio as asyncio
import wifi
import send_data
import sensors
import gc
import collections.deque
import ntptime
import network
import utime
import machine
import power_status

print("running garbage collector")
gc.collect()  # clean up memory after imports

wifi_interface = network.WLAN(network.STA_IF)

while not wifi_interface.isconnected():
    wifi.connectWifi()

print('network config:', wifi_interface.ifconfig())

data_queue = collections.deque.deque()
get_ID = send_data.generateMessageID()

print("setting time")
ntptime.settime()

def wdt_callback():
    global wdt_counter
    wdt_counter += 1
    print("wdt counter set to: {}".format(wdt_counter))
    if (wdt_counter > 10):
        try:
            print("watchdog timer has timed out")
            send_data.postData({'reset_time_date': list(utime.localtime()),
                                'reset_cause': 'watchdog_timer'}, 
                               url="http://192.168.1.151:9494/error")
        except Exception as e:
            print("Error sending wdt data: {}".format(e))
        finally:
            print("resetting device")
            machine.reset()

wdt_counter = 0
# Set a virtual timer by setting Timer ID of -1
wdt_timer = machine.Timer(-1)
# Create a periodic timer with period of 60000ms, aka 60 seconds
wdt_timer.init(period=60000, mode=machine.Timer.PERIODIC, callback=lambda t:wdt_callback())
print("wdt initialized")

# Reset codes:
# 0 = power on, 1 = WDT reset, 4 = soft reset, 5 = DEEP_SLEEP reset, 6 = hard reset,
def reportLastReset():
    try:
        print("Sending reset data")
        send_data.postData({'reset_time_date': list(utime.localtime()),
                            'reset_cause': machine.reset_cause()}, 
                            url="http://192.168.1.151:9494/error")
        print("Sent reset data to server")
    except Exception as e:
            print("Error sending reset data: {}".format(e))

reportLastReset()

loop_counter = 0

while True:
    print("loop counter = ", loop_counter)
    
    print("getting data")
    data_queue.append(sensors.readData())
    
    if loop_counter % 3 == 0:
        print("sending data")
        send_data.processQueue(data_queue, get_ID, "http://192.168.1.151:9494/station")
        print("feeding WatchDog timer")
        wdt_counter = 0

    if loop_counter % 6 == 0:
        print("checking Wifi Connection")
        wifi.connectWifi()
        print("sending battery status")
        send_data.postData(power_status.getBatteryStatus(),
                           url="http://192.168.1.151:9494/power")
    
    if loop_counter % 30 == 0:
        print("sending log data")
        #TODO send memory info, queue length
        
    if loop_counter % 60 == 0:
        print("collecting garbage")
        #http://docs.micropython.org/en/latest/reference/constrained.html#the-heap
        gc.collect()
        gc.threshold(gc.mem_free() // 4 + gc.mem_alloc())
        
    if loop_counter % 2000 == 0:
        print("setting time with ntp")
        try:
            ntptime.settime()
        except OSError as e:
            print(e)

    loop_counter += 1
    if loop_counter > 2000:
        loop_counter = 0

    utime.sleep(10) # Consider deep sleep modes - https://docs.micropython.org/en/latest/library/esp.html
"""

In [ ]:
os.listdir()

In [ ]:
with open('main.py') as f:
    print(f.read())

In [ ]:
os.remove('main.py')

In [ ]:
with open('main.txt', 'w') as f:
    f.write(main_script)

### Double check the write

In [ ]:
with open('main.txt') as f:
    print(f.read())

## Move temp file to `send_data.py`

In [ ]:
os.rename('main.txt', 'main.py')

## Simple software WDT implementation from https://forum.micropython.org/viewtopic.php?f=16&t=5517&start=10
```
wdt_counter = 0

def wdt_callback():
    global wdt_counter
    wdt_counter += 1
    if (wdt_counter >= 10):
        machine.reset()

def wdt_feed():
    global wdt_counter
    wdt_counter = 0

wdt_timer = machine.Timer(-1)
wdt_timer.init(period=1000, mode=machine.Timer.PERIODIC, callback=lambda t:wdt_callback())
## END Simple software WDT implementation
```

In [ ]:
%rebootdevice

In [ ]:
%disconnect